In [507]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.cluster import hierarchy
import tensorflow as tf
from sklearn.model_selection import train_test_split

In [508]:
%matplotlib widget
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.activations import relu,linear
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.optimizers import Adam

In [509]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.activations import linear, relu, sigmoid, softmax

In [510]:
df = pd.read_csv('Train_Data_Final.csv')

In [523]:
df1 = df[df['AH'] >= 0]
df1 = df1.drop(['Date', 'Time','id','NMHC(GT)'], axis=1)

In [524]:
df1.columns

Index(['CO(GT)', 'PT08.S1(CO)', 'C6H6(GT)', 'PT08.S2(NMHC)', 'NOx(GT)',
       'PT08.S3(NOx)', 'NO2(GT)', 'PT08.S4(NO2)', 'PT08.S5(O3)', 'T', 'AH',
       'RH_type'],
      dtype='object')

In [525]:
df_ = df1.replace(-200, np.nan)

In [526]:
df_ = df_.drop(['RH_type'], axis = 1)

In [527]:
from sklearn.impute import KNNImputer

imputer = KNNImputer(n_neighbors=5)
df_imputed = imputer.fit_transform(df_)
df_imputed = pd.DataFrame(df_imputed, columns=df_.columns)
df_ = df_imputed


In [528]:
rows_with_nan = df_.isnull().any(axis=1)
df_nan = df_[rows_with_nan]
df_nan

,CO(GT),PT08.S1(CO),C6H6(GT),PT08.S2(NMHC),NOx(GT),PT08.S3(NOx),NO2(GT),PT08.S4(NO2),PT08.S5(O3),T,AH


In [529]:
df_normalized = df_.div(df_.max())

In [530]:
df_ = df_normalized
df_normalized.describe()

,CO(GT),PT08.S1(CO),C6H6(GT),PT08.S2(NMHC),NOx(GT),PT08.S3(NOx),NO2(GT),PT08.S4(NO2),PT08.S5(O3),T,AH
count,7711.000000,7711.000000,7711.000000,7711.000000,7711.000000,7711.000000,7711.000000,7711.000000,7711.000000,7711.000000,7711.000000
mean,0.175317,0.538362,0.164041,0.430605,0.148497,0.316354,0.306868,0.543077,0.405886,0.436564,0.484933
std,0.120753,0.107828,0.118920,0.121076,0.135655,0.096775,0.130991,0.119454,0.156163,0.194284,0.176952
min,0.008403,0.317318,0.002338,0.173103,0.001352,0.120026,0.006013,0.236667,0.100089,0.007287,0.089087
25%,0.084034,0.456714,0.073922,0.338189,0.055443,0.250303,0.207456,0.470270,0.292241,0.293161,0.365903
50%,0.142857,0.520039,0.135307,0.417570,0.104124,0.304818,0.297354,0.543423,0.381231,0.432175,0.469070
75%,0.235294,0.604731,0.225900,0.510163,0.191278,0.366974,0.383343,0.615180,0.504096,0.568853,0.614647
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [531]:
X_train, X_test, y_train, y_test = train_test_split(df_, df1['RH_type'], train_size=0.75, random_state=0)

In [532]:
y_train.replace({"Dry": 0, "Ideal": 1, "Slightly Elevated": 2, "Elevated": 3, "High": 4}, inplace=True)
y_train = np.array(y_train).reshape(-1, 1) 
y_test.replace({"Dry": 0, "Ideal": 1, "Slightly Elevated": 2, "Elevated": 3, "High": 4}, inplace=True)
y_test = np.array(y_test).reshape(-1, 1) 
X_train = X_train.to_numpy()
X_test = X_test.to_numpy()

In [533]:
model = Sequential(
    [             
        Dense(units = 50, activation = 'relu'),
        Dense(units = 5, activation = 'softmax')
    ], name = "my_model" 
)
model.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.01)
)

history = model.fit(
    X_train,y_train,
    epochs=500
)

Epoch 1/500
181/181 ━━━━━━━━━━━━━━━━━━━━ 1s 460us/step - loss: 1.3999 
Epoch 2/500
181/181 ━━━━━━━━━━━━━━━━━━━━ 0s 438us/step - loss: 0.7958
Epoch 3/500
181/181 ━━━━━━━━━━━━━━━━━━━━ 0s 447us/step - loss: 0.5904
Epoch 4/500
181/181 ━━━━━━━━━━━━━━━━━━━━ 0s 452us/step - loss: 0.4857
Epoch 5/500
181/181 ━━━━━━━━━━━━━━━━━━━━ 0s 453us/step - loss: 0.4405
Epoch 6/500
181/181 ━━━━━━━━━━━━━━━━━━━━ 0s 455us/step - loss: 0.3818
Epoch 7/500
181/181 ━━━━━━━━━━━━━━━━━━━━ 0s 467us/step - loss: 0.3457
Epoch 8/500
181/181 ━━━━━━━━━━━━━━━━━━━━ 0s 462us/step - loss: 0.3195
Epoch 9/500
181/181 ━━━━━━━━━━━━━━━━━━━━ 0s 465us/step - loss: 0.3069
Epoch 10/500
181/181 ━━━━━━━━━━━━━━━━━━━━ 0s 464us/step - loss: 0.2943
Epoch 11/500
181/181 ━━━━━━━━━━━━━━━━━━━━ 0s 451us/step - loss: 0.2692
Epoch 12/500
181/181 ━━━━━━━━━━━━━━━━━━━━ 0s 450us/step - loss: 0.2628
Epoch 13/500
181/181 ━━━━━━━━━━━━━━━━━━━━ 0s 442us/step - loss: 0.2471
Epoch 14/500
181/181 ━━━━━━━━━━━━━━━━━━━━ 0s 441us/step - loss: 0.2345
Epoch 15/500
1

In [536]:
prediction = model.predict(X_test)
prediction
yhat = np.array(prediction.argmax(axis=1))
yhat = np.array(yhat).reshape(-1,1)
acc = 0
for i in range(yhat.shape[0]):
    if(yhat[i] == y_test[i]):
        acc+=1
acc/=yhat.shape[0]
acc

61/61 ━━━━━━━━━━━━━━━━━━━━ 0s 556us/step


0.9761410788381742